In [19]:
import tensorflow as tf

# Load the Keras model
keras_model = tf.keras.models.load_model('bees-wasps.h5')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmpsfizcto1/assets


INFO:tensorflow:Assets written to: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmpsfizcto1/assets
2023-11-29 18:46:04.680576: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-29 18:46:04.680586: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-29 18:46:04.680691: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmpsfizcto1
2023-11-29 18:46:04.681295: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-29 18:46:04.681299: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/22/4rkd_km14n587vm5mk020x800000gq/T/tmpsfizcto1
2023-11-29 18:46:04.682736: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-29 18:46:04.788754: I tensorflow/cc/saved_model/loader.cc:217] Running initialization

In [20]:
# # Load the TFLite model and allocate tensors.
# interpreter = tf.lite.Interpreter(model_path="model.tflite")
# interpreter.allocate_tensors()

# # Get output details
# output_details = interpreter.get_output_details()

# # Print the index of each output
# for output in output_details:
#     print(output['index'])

In [21]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [22]:
img = Image.open('/Users/inristaka/dev/machine-learning-zoomcamp/cohorts/2023/09-serverless/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')

In [23]:
img = img.resize([150,150], Image.NEAREST)
print(img.size)

(150, 150)


In [24]:
import numpy as np

In [25]:
def preprocess_input(x):
    x /= 255.
    return x
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [26]:
X

array([[[[0.94509804, 0.9098039 , 0.8509804 ],
         [0.9372549 , 0.92156863, 0.9764706 ],
         [0.9137255 , 0.8980392 , 0.95686275],
         ...,
         [0.2901961 , 0.33333334, 0.16470589],
         [0.34901962, 0.40784314, 0.15294118],
         [0.29803923, 0.36078432, 0.11764706]],

        [[0.9490196 , 0.9098039 , 0.8627451 ],
         [0.91764706, 0.9098039 , 0.9607843 ],
         [0.9019608 , 0.89411765, 0.9490196 ],
         ...,
         [0.27450982, 0.3372549 , 0.16078432],
         [0.4627451 , 0.5058824 , 0.19607843],
         [0.4509804 , 0.49411765, 0.18431373]],

        [[0.92941177, 0.88235295, 0.81960785],
         [0.9137255 , 0.9098039 , 0.9647059 ],
         [0.90588236, 0.89411765, 0.9607843 ],
         ...,
         [0.32156864, 0.3764706 , 0.1764706 ],
         [0.49803922, 0.5294118 , 0.20392157],
         [0.43137255, 0.46666667, 0.21176471]],

        ...,

        [[0.03137255, 0.06666667, 0.04705882],
         [0.02352941, 0.08627451, 0.04705882]

In [27]:
import tensorflow.lite as tflite

In [28]:
interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [29]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [30]:
preds

array([[0.65921456]], dtype=float32)

In [31]:
classes = [
    'bees',
    'wasps'
]

dict(zip(classes, preds[0]))

{'bees': 0.65921456}